# <center>Workshop de Data Science</center>
# <center>Introdução a NLP</center>
# <center>Visagio</center>
___

# 0 Introdução

Iremos importar todas as bibliotecas necessárias para os exercícios da aula

In [ ]:
# Importe as bibliotecas necessárias
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

!pip install -q fuzzywuzzy dataprep
from fuzzywuzzy import fuzz
from dataprep.clean import clean_text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

# Carregue o arquivo de dados
data = pd.read_excel('/content/vacina_tweets_sentiment.xlsx', usecols=['text', 'len_text', 'sentiment', 'vacina', 'conf_positive', 'conf_negative', 'conf_neutral', 'conf_mixed'])
data.head()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.0/764.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 55.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,text,vacina,len_text,sentiment,conf_positive,conf_negative,conf_neutral,conf_mixed
0,O programa está ótimo como sempre! Parabéns me...,russia,89,POSITIVE,0.999785,0.000020,0.000194,0.000001
1,@mateusnjs Vacina lá chegou antes que na Rússi...,russia,82,POSITIVE,0.999400,0.000028,0.000570,0.000002
2,Hoje era um dia PERFEITO pra sentar minha raba...,russia,140,POSITIVE,0.999070,0.000065,0.000864,0.000001
3,A cada mensagem de Feliz Aniversário eu fico m...,oxford,140,POSITIVE,0.998115,0.000053,0.001825,0.000007
4,RT @marciaturcato: O programa está ótimo como ...,russia,108,POSITIVE,0.997563,0.000205,0.002230,0.000002


# 1 Correspondência de texto (Text Matching)




## 1.1 Regex

Vamos começar com regex. Use a biblioteca re do Python para realizar as seguintes tarefas:

1. Encontre todos os tweets que mencionam "Pfizer" na coluna 'text'.
2. Encontre todos os tweets que mencionam hashtags (#) na coluna 'text'.

In [ ]:
# Encontre todos os tweets que mencionam "Pfizer" na coluna 'text'
# Dica: utilize o método str.contains(r'',regex=True) para realizar o filtro de linhas

pattern = r'Pfizer'
pfizer_tweets = data[data['text'].str.contains(pattern, regex=True, flags = re.IGNORECASE)]

print(pfizer_tweets)

                                                    text  vacina  len_text  \
5554   RT @hfernandesbio: Quais são as mais promissor...  oxford       140   
5921   @MonicaDireita @MahggaSantana É... falei pra e...  oxford       118   
7220   Quais são as mais promissoras? #vacina #COVID1...  oxford       139   
8116   Precisamos dessa vacina russa para ontem. Li u...  oxford       140   
9184   @PotamusGloria Será assim com a vacina de Oxfo...  oxford       139   
10016  @titimuller Nope 😰. A gente não tá gritando pe...  oxford       140   
11403  @0800ree @RenanPeixoto_ @g_bettio Trump compro...  oxford       140   

      sentiment  conf_positive  conf_negative  conf_neutral  conf_mixed  
5554    NEUTRAL       0.020311       0.002615      0.977056    0.000018  
5921    NEUTRAL       0.012064       0.203760      0.784175    0.000002  
7220    NEUTRAL       0.002945       0.000059      0.996988    0.000008  
8116    NEUTRAL       0.002395       0.000049      0.997554    0.000001  
9184 

In [ ]:
# Encontre todos os tweets que mencionam hashtags (#) na coluna 'text'

pattern = r'#'
hashtag_tweets = data[data['text'].str.contains(pattern, regex=True)]

print(hashtag_tweets)

                                                    text  vacina  len_text  \
269    Amanha nao esqueçam usem a masacarazinha \n......  oxford       139   
311    Seja de Oxford seja Russa ou da China eu tô pr...  oxford        75   
312    RÚSSIA VAMO SOLTA ESSA VACINA AE Q AINDA DA TE...  russia        76   
785    RT @PeopleMary_: Roi, Rússia né? Obrigada por ...  russia        91   
795    RT @ariquemesonline: Ministro Interino Eduardo...  oxford       140   
...                                                  ...     ...       ...   
12028  Rússia anuncia produção de primeiro lote de va...  russia        82   
12034  Hoje tem #Neymar? E daí? Precisamos é de vacin...  oxford       134   
12053  Rússia inicia produção da vacina contra o coro...  russia       131   
12073  México e Argentina só não produzirão vacina de...  oxford       104   
12089  #JornalPrimeiraHora: A Rússia já está repassan...  russia       140   

      sentiment  conf_positive  conf_negative  conf_neutral  co

## 1.2 Fuzzy Text Matching

Agora, trabalhando com correspondência difusa. Use a biblioteca fuzzywuzzy ou difflib para:

1. Encontrar tweets semelhantes aos tweets que mencionam "Moderna" na coluna 'text'.
2. Calcular a pontuação de similaridade de Levenshtein entre dois tweets quaisquer na coluna 'text'.

In [ ]:
# Encontre tweets semelhantes aos tweets que mencionam "Moderna" na coluna 'text'
# Utilize a função fuzz.token_set_ratio combinado com o método apply

# Filtrando tweets que falam da vacina "Moderna"
pattern = r'Moderna'
moderna_tweets = data[data['text'].str.contains(pattern, flags = re.IGNORECASE, regex=True)]

# Encontrando tweets similares
similar_tweets = data.text.apply(lambda x: fuzz.token_set_ratio(moderna_tweets.iloc[0], x))

print(similar_tweets)

0        27
1        30
2        25
3        24
4        35
         ..
12093    24
12094    27
12095    24
12096    30
12097    36
Name: text, Length: 12098, dtype: int64


In [ ]:
# Calcule a pontuação de similaridade de Levenshtein entre dois tweets quaisquer na coluna 'text'
tweet1 = data['text'].iloc[0]
tweet2 = data['text'].iloc[1]
levenshtein_score = fuzz.ratio(tweet1, tweet2)
levenshtein_score

30

# 2 Pré-processamento de Dados

Usando a biblioteca dataprep, realize as seguintes tarefas:

1. Limpe os tweets na coluna 'text' removendo caracteres especiais, emojis e espaços extras.
2. Calcule estatísticas descritivas dos tamanhos dos textos na coluna 'len_text'.

In [ ]:
# Limpe os tweets na coluna 'text'
# Dica: Use a função clean_text da biblioteca dataprep: https://docs.dataprep.ai/user_guide/clean/clean_text.html
# Remova as stopwords presentes no corpus set(stopwords.words('portuguese'))
cleaned_text =
cleaned_text.head()

,text,vacina,len_text,sentiment,conf_positive,conf_negative,conf_neutral,conf_mixed
0,O programa está ótimo como sempre! Parabéns me...,russia,89,POSITIVE,0.999785,0.000020,0.000194,0.000001
1,@mateusnjs Vacina lá chegou antes que na Rússi...,russia,82,POSITIVE,0.999400,0.000028,0.000570,0.000002
2,Hoje era um dia PERFEITO pra sentar minha raba...,russia,140,POSITIVE,0.999070,0.000065,0.000864,0.000001
3,A cada mensagem de Feliz Aniversário eu fico m...,oxford,140,POSITIVE,0.998115,0.000053,0.001825,0.000007
4,RT @marciaturcato: O programa está ótimo como ...,russia,108,POSITIVE,0.997563,0.000205,0.002230,0.000002


In [ ]:
# Calcule estatísticas descritivas dos tamanhos dos textos na coluna 'len_text'
# Dica: Utilize o método .describe()
text_length_stats = __
text_length_stats

count    12098.000000
mean       126.993222
std         21.829048
min         26.000000
25%        119.000000
50%        140.000000
75%        140.000000
max        144.000000
Name: len_text, dtype: float64

# 3 Train Test Split
Dando início à etapa de modelagem e engenharia de variáveis

1. Divida os dados em conjuntos de treinamento e teste.

In [ ]:
# Divida os dados em conjuntos de treinamento e teste
# Use a função train_test_split: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X = cleaned_text['text']
y = cleaned_text['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4 Texto como Features
Agora, trabalhando com texto como recursos, use as bibliotecas sklearn e nltk para:

1. Criar uma matriz de "Bag of Words" (BoW) usando a coluna 'text' do novo dataframe 'cleaned_text'.
2. Calcular as pontuações TF-IDF para os tweets na coluna 'text' do novo dataframe 'cleaned_text'.

In [ ]:
# Crie uma matriz de "Bag of Words" (BoW) usando a coluna 'text'
# Dica: Use os métodos .fit_transform(X) para o treino e .transform(X) para o teste

vectorizer = CountVectorizer()
bow_matrix_train = __
bow_matrix_test = __

In [ ]:
# Calcule as pontuações TF-IDF para os tweets na coluna 'text'
# Dica: Use os métodos .fit_transform(X) para o treino e .transform(X) para o teste
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_train = __
tfidf_matrix_test = __

# 5 Análise de Sentimentos
Finalmente, vamos construir um modelo de análise de sentimentos usando scikit-learn ou NLTK. Faça o seguinte:

1. Crie um modelo de análise de sentimentos para prever a coluna 'sentiment' com base no texto.
2. Avalie o desempenho do modelo usando métricas como precisão, recall e F1-score.

In [ ]:
# Crie um modelo de análise de sentimentos
# Dica: Preencha as lacunas com os dados de treino e teste criados anteriormente

classifier = MultinomialNB()
classifier.fit(__,__)
y_pred = classifier.predict(__)

In [ ]:
# Avalie o desempenho do modelo
accuracy = accuracy_score(__,__)
classification_report_str = classification_report(__, __)

print(accuracy)
print(classification_report_str)